In [ ]:
from ultralytics import YOLO
import cv2

def run_webcam_detection(model_path: str) -> None:
    # Load the YOLO model from the given path.
    model: YOLO = YOLO(model_path)
    
    # Open the default webcam.
    cap: cv2.VideoCapture = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the webcam.")
        return

    # Attempt to set the webcam resolution to 4K (3840x2160).
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    
    # Optional: Print the current resolution to verify.
    width: float = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height: float = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print(f"Webcam resolution set to: {width} x {height} 📸")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run the YOLO model on the current frame.
        results = model(frame)
        
        # Annotate the frame with the detection results.
        annotated_frame = results[0].plot()
        
        # Display the annotated frame.
        cv2.imshow("YOLO Detection", annotated_frame)
        
        # Exit the loop if 'q' is pressed.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window.
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_webcam_detection("runs/detect/train-OD11m-v6/weights/best.pt")

In [ ]:
from ultralytics import YOLO
import cv2

def run_webcam_tracking(model_path: str) -> None:
    # Load the YOLO model
    model: YOLO = YOLO(model_path)
    
    # Open the webcam
    cap: cv2.VideoCapture = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the webcam.")
        return

    # Set webcam resolution to 4K (3840x2160)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    
    # Print the current resolution to verify
    width: float = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height: float = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print(f"Webcam resolution set to: {width} x {height} 📸")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLO with ByteTrack enabled
        results = model.track(frame, persist=True, tracker="bytetrack.yaml")
        
        # Annotate the frame with detection and tracking results
        annotated_frame = results[0].plot()
        
        # Display the annotated frame
        cv2.imshow("YOLO + ByteTrack", annotated_frame)
        
        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_webcam_tracking("runs/detect/train-OD11m-v6/weights/best.pt")


Webcam resolution set to: 1552.0 x 1552.0 📸

0: 1088x1088 (no detections), 839.2ms
Speed: 6.8ms preprocess, 839.2ms inference, 0.3ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 782.9ms
Speed: 6.4ms preprocess, 782.9ms inference, 0.4ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 728.4ms
Speed: 5.9ms preprocess, 728.4ms inference, 0.2ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 773.1ms
Speed: 6.1ms preprocess, 773.1ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 757.3ms
Speed: 6.2ms preprocess, 757.3ms inference, 0.4ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 742.2ms
Speed: 5.4ms preprocess, 742.2ms inference, 0.3ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 747.6ms
Speed: 5.6ms preprocess, 747.6ms inference, 0.4ms postprocess per image at shape (1

2025-03-04 10:47:48.340 python[78219:5126865] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-04 10:47:48.340 python[78219:5126865] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 1088x1088 (no detections), 713.6ms
Speed: 5.6ms preprocess, 713.6ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 700.5ms
Speed: 5.8ms preprocess, 700.5ms inference, 0.3ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 693.5ms
Speed: 5.8ms preprocess, 693.5ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 704.9ms
Speed: 6.2ms preprocess, 704.9ms inference, 0.2ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 720.0ms
Speed: 5.2ms preprocess, 720.0ms inference, 0.3ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 741.0ms
Speed: 5.1ms preprocess, 741.0ms inference, 0.5ms postprocess per image at shape (1, 3, 1088, 1088)

0: 1088x1088 (no detections), 794.6ms
Speed: 5.6ms preprocess, 794.6ms inference, 0.3ms postprocess per image at shape (1, 3, 1088, 1088)

